# __Automation experiment__

In [ ]:
from kubernetes import client, config
from kubernetes.client.rest import ApiException
import yaml
import boto3

# __Amazon EKS: node group management__

### connect to eks client

In [ ]:
def connect_eks_client(aws_region):
    return boto3.client('eks', region_name=aws_region)

### set variables for eks

In [ ]:
class EKS_variable:
    def __init__(self):
        self.aws_region = 'us-east-1'
        self.cluster_name = 'cluster-tanimura'
        self.nodegroup_name = 'test_nodegroup'
        self.subnets = ['subnet-00db610ac559c4a1a', 'subnet-0b90c6fe07ca5621e']
        self.instance_types = 't3.medium'
        self.ami_type = 'AL2_x86_64'
        self.ssh_key_name = 'ec2-key-dong'
        self.node_role_arn = 'arn:aws:iam::972269765820:role/AmazonEKSNodeRole'
        self.min_size = 2
        self.max_size = 2
        self.desired_size = 2
        self.disk_size = 20

### create node group

In [ ]:
def create_set_nodegroup(eks_client, var):
    response = eks_client.create_nodegroup(
        clusterName=var.cluster_name,
        nodegroupName=var.nodegroup_name,
        scalingConfig={
            'minSize': var.min_size,
            'maxSize': var.max_size,
            'desiredSize': var.desired_size
        },
        diskSize=var.disk_size,
        subnets=var.subnets,
        instanceTypes=var.instance_types,
        amiType=var.ami_type,
        remoteAccess={
            'ec2SshKey': var.ssh_key_name
        },
        nodeRole=var.node_role_arn
    )

    return response

In [ ]:
def create_nodegroup():
    eks_var = EKS_variable()

    # connect to eks client
    eks_client = connect_eks_client(eks_var.aws_region)

    # create Nodegroup
    response = create_set_nodegroup(eks_client, eks_var)

    return response

### delete node group

In [ ]:
def delete_set_nodegroup(eks_client, var):
    response = eks_client.delete_nodegroup(
        clusterName=var.cluster_name,
        nodegroupName=var.nodegroup_name
    )

    return response

In [ ]:
def delete_nodegroup():
    eks_var = EKS_variable()

    # connect to eks client
    eks_client = connect_eks_client(eks_var.aws_region)

    # delete Nodegroup
    response = delete_set_nodegroup(eks_client, eks_var)

    return response

# __MQTT broker and clients__

### detailed error messages

In [ ]:
def api_exception(exception):
    # output detailed error messages for ApiException
    print(f"Exception: {exception}")
    print(f"Status code: {exception.status}")
    print(f"Response body: {exception.body}")
    print(f"Reason: {exception.reason}")
    print(f"Headers: {exception.headers}")

### set variables for MQTT

In [ ]:
class MQTT_variable:
    def __init__(self):
        self.relay_broker = 'broker_secrets.yaml'
        self.receiving_broker = ''
        self.publisher = 'publisher_template.yaml'
        self.subscriber = ''
        self.subpub = ''
        self.namespace = 'default'
        self.replicas = 10
        self.req_size = '102372'
        self.exp_time = '0.1'


# MQTT broker

### create relay brokers 

In [ ]:
def create_relay_broker():
    var = MQTT_variable()
    try:
        # load kubeconfig
        config.load_kube_config()
        # create client
        apps_v1_api = client.AppsV1Api()

        with open(var.relay_broker) as f:
            daemonset_manifest = yaml.safe_load(f)

        broker_namespace = var.namespace
        # create DaemonSet
        apps_v1_api.create_namespaced_daemon_set(namespace=broker_namespace, body=daemonset_manifest)
    except ApiException as e:
        # output detailed error messages
        api_exception(e)
        return []

### Get broker name

In [ ]:
def get_broker_names(var):
    try:
        # load kubeconfig
        config.load_kube_config()
        # create client
        v1_core_api = client.CoreV1Api()

        broker_namespace = var.namespace
        # get pod list
        pods_list = v1_core_api.list_namespaced_pod(broker_namespace, watch=False)
        # get pod name
        broker_names = [pod.metadata.name for pod in pods_list.items]
        
        return broker_names
    except ApiException as e:
        api_exception(e)
        return []

### Get pod's ip address

In [ ]:
def get_pod_ip(pod_name, var):
    try:
        # load kubeconfig
        config.load_kube_config()
        # create client
        v1_core_api = client.CoreV1Api()

        broker_namespace = var.namespace
        # get pod's ip
        pod = v1_core_api.read_namespaced_pod(pod_name, broker_namespace)
        pod_ip = pod.status.pod_ip

        return pod_ip
    except ApiException as e:
        api_exception(e)
        return None

### Get pod's node name

In [ ]:
def get_pod_nodename(pod_name, var):
    try:
        # load kubeconfig
        config.load_kube_config()
        # create client
        v1_core_api = client.CoreV1Api()

        broker_namespace = var.namespace
        # get pod's node name
        pod = v1_core_api.read_namespaced_pod(pod_name, broker_namespace)
        pod_nodename = pod.spec.node_name

        return pod_nodename
    except ApiException as e:
        api_exception(e)
        return None

### Save pod's ip and node name

In [ ]:
def save_info_to_dict(broker_names):
    var = MQTT_variable()
    pod_ip_dict = {}
    pod_nodename_dict = {}

    for name in broker_names:
        ip = get_pod_ip(name, var)
        nodename = get_pod_nodename(name, var)
        
        if ip and nodename:
            pod_ip_dict[name] = ip
            pod_nodename_dict[name] = nodename

    return pod_ip_dict, pod_nodename_dict

# MQTT clients

### Load client template

In [ ]:
def load_template(yaml_file):
    with open(yaml_file, 'r') as file:
        template_content = file.read()
        return yaml.safe_load(template_content)

### Change the noda name and broker ip for publisher to create it on each broker's node

In [ ]:
def publisher_deployment(nodename, broker_ip):
    try:
        # load kubeconfig
        config.load_kube_config()
        # create client
        apps_v1_api = client.AppsV1Api()

        template = load_template()
        # change node name and broker ip
        template.spec.template.spec.node_name = nodename
        template.spec.template.spec.containers[0].env[0].value = broker_ip

        namespace = "default"
        # create Deployment
        apps_v1_api.create_namespaced_deployment(namespace, body=template)
    except ApiException as e:
        api_exception(e)